In [3]:
import torch
import torch.nn as nn
from networks import DynamicUNet

network = DynamicUNet(1, 3, [64, 128])
additional_pass = network.make_additional_step(256)

#def calculate_train_loss(self, X, y, additional_step = None, loss_fn=nn.CrossEntropyLoss(), alpha = 1.0):

loss = network.calculate_train_loss(torch.randn((5, 1, 64, 64)), torch.randn((5, 1, 64, 64)), additional_pass, alpha=0.5, loss_fn = nn.MSELoss())

torch.Size([5, 256, 16, 16]) torch.Size([5, 256, 16, 16])


/usr/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([5, 1, 64, 64])) that is different to the input size (torch.Size([5, 3, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [4]:
from torchviz import make_dot

make_dot(loss, params=dict(list(network.named_parameters()))).render("torchviz", format="png")

'torchviz.png'